## 156: Registro de usuarios y almacenamiento en memoria con FastAPI 

Agregar funcionalidad de registro de usuarios y almacenamiento en memoria, simulando una base de datos, junto con la autenticación y manejo de roles usando JWT.

🧩 Código completo: Registro, Login y Autenticación con Roles
python
Copiar
Editar
from fastapi import FastAPI, HTTPException, Depends
from fastapi.security import OAuth2PasswordBearer
from pydantic import BaseModel
from typing import Optional, Dict
from datetime import datetime, timedelta
from jose import jwt, JWTError
from passlib.context import CryptContext

# Configuración básica
app = FastAPI()
SECRET_KEY = "CLAVE_ULTRA_SECRETA"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="login")
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# Base de datos simulada
usuarios_db: Dict[str, Dict] = {}

# Modelos
class UsuarioRegistro(BaseModel):
    email: str
    contraseña: str
    rol: str

class UsuarioLogin(BaseModel):
    email: str
    contraseña: str

# Utilidades
def hashear_contraseña(contraseña: str) -> str:
    return pwd_context.hash(contraseña)

def verificar_contraseña(contraseña: str, hash: str) -> bool:
    return pwd_context.verify(contraseña, hash)

def crear_token(email: str, rol: str) -> str:
    expiracion = datetime.utcnow() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    datos = {"sub": email, "rol": rol, "exp": expiracion}
    return jwt.encode(datos, SECRET_KEY, algorithm=ALGORITHM)

def obtener_usuario_actual(token: str = Depends(oauth2_scheme)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        email = payload.get("sub")
        rol = payload.get("rol")
        if email is None or rol is None:
            raise HTTPException(status_code=401, detail="Token inválido")
        return {"email": email, "rol": rol}
    except JWTError:
        raise HTTPException(status_code=401, detail="Token inválido o expirado")

# Rutas
@app.post("/registrar")
def registrar(usuario: UsuarioRegistro):
    if usuario.email in usuarios_db:
        raise HTTPException(status_code=400, detail="El usuario ya existe")
    usuarios_db[usuario.email] = {
        "contraseña": hashear_contraseña(usuario.contraseña),
        "rol": usuario.rol
    }
    return {"mensaje": "Usuario registrado exitosamente"}

@app.post("/login")
def login(datos: UsuarioLogin):
    usuario = usuarios_db.get(datos.email)
    if not usuario or not verificar_contraseña(datos.contraseña, usuario["contraseña"]):
        raise HTTPException(status_code=401, detail="Credenciales inválidas")
    token = crear_token(datos.email, usuario["rol"])
    return {"access_token": token, "token_type": "bearer"}

@app.get("/admin")
def solo_admin(usuario=Depends(obtener_usuario_actual)):
    if usuario["rol"] != "admin":
        raise HTTPException(status_code=403, detail="Acceso denegado")
    return {"mensaje": f"Bienvenido, administrador {usuario['email']}"}

@app.get("/perfil")
def perfil(usuario=Depends(obtener_usuario_actual)):
    return {"email": usuario["email"], "rol": usuario["rol"]}